In [53]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

import pandas as pd

import timeit

from IPython.display import display

In [12]:
class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset. 
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from
    """
    def __init__(self, num_samples, start = 0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples

In [41]:
# Load data
train_path = './input/train-jpg/'
test_path = './input/test-jpg/'
train = pd.read_csv('./input/train_v2.csv')
test = pd.read_csv('./input/sample_submission_v2.csv')

In [42]:
print(train.shape)
print(test.shape)
NUM_TRAIN = 32000
NUM_VAL = train.shape[0]-NUM_TRAIN
NUM_TEST = test.shape[0]

(40479, 2)
(61191, 2)


In [43]:
print(train.iloc[0])
print(test.iloc[0])

image_name         train_0
tags          haze primary
Name: 0, dtype: object
image_name                                  test_0
tags          primary clear agriculture road water
Name: 0, dtype: object


In [55]:
display(train[0:10])

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road
5,train_5,haze primary water
6,train_6,agriculture clear cultivation primary water
7,train_7,haze primary
8,train_8,agriculture clear cultivation primary
9,train_9,agriculture clear cultivation primary road


In [57]:
train['tags'][0].split()

['haze', 'primary']

TBD
Make loop to import all images & store as a numpy array of (32,32,3)'s
Save extracted image data somewhere so I don't need to preprocess each time
Convert text labels into multi-hot vectors, with vocab as the 17 labels in alphabetical order. 1 = agriculture, 2 = clear, etc.